In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

In [20]:
glogs_final = pd.read_csv('https://raw.githubusercontent.com/tmarchok1/DS440_project/refs/heads/main/glogs_final.csv')
glogs_final.head()

,date,year,week,day_of_week,v_name,h_name,day_night,park_id,temp,precip,capacity,prev_year_wins,made_playoffs,won_division,InstagramFollowers,CityPopulation,attendance
0,2005-04-03,2005,1,Sun,BOS,NYY,1,Old Yankee Stadium,51.7,0.587,56937.0,101,1,1,3900000,19940274,54818.0
1,2005-04-04,2005,1,Mon,OAK,BAL,0,Camden Yards,60.7,0.000,45971.0,78,0,0,746000,2859024,48271.0
2,2005-04-04,2005,1,Mon,CLE,CHW,0,US Cellular Field,56.2,0.000,40615.0,83,0,0,664000,9408576,38141.0
3,2005-04-04,2005,1,Mon,KC,DET,0,Comerica Park,59.0,0.000,41083.0,72,0,0,1000000,4400587,44105.0
4,2005-04-04,2005,1,Mon,MIN,SEA,0,Safeco Field,50.1,0.130,47943.0,63,0,0,927000,4145494,46249.0


In [21]:
glogs_final = pd.read_csv('https://raw.githubusercontent.com/tmarchok1/DS440_project/refs/heads/main/glogs_final.csv')
glogs_final.head()

# Drop columns
glogs_final = glogs_final.drop(columns=['date', 'year', 'temp'])

# Binary encode 'day_of_week' and 'precip'
glogs_final['day_of_week'] = glogs_final['day_of_week'].map({'Mon':0, 'Tue':0, 'Wed':0, 'Thu':0, 'Fri':1, 'Sat':1, 'Sun':1}).astype('int')
glogs_final['precip'] = glogs_final['precip'].map(lambda x: 1 if x != 0 else 0)
glogs_final

# Apply OneHotEncoder to 'v_name', 'h_name', 'park_id' columns
encoder = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(sparse_output=False), ['week', 'v_name', 'h_name', 'park_id'])],
    remainder='passthrough'  # Keep other columns if there are any
)

df_encoded = encoder.fit_transform(glogs_final)

# Rename columns
feature_names = encoder.get_feature_names_out()
feature_names = [name.replace("remainder__", "") for name in feature_names]

# Convert the result back to a DataFrame (optional)
df_encoded = pd.DataFrame(df_encoded, columns=feature_names)
df_encoded

,cat__week_1,cat__week_2,cat__week_3,cat__week_4,cat__week_5,cat__week_6,cat__week_7,cat__week_8,cat__week_9,cat__week_10,...,day_of_week,day_night,precip,capacity,prev_year_wins,made_playoffs,won_division,InstagramFollowers,CityPopulation,attendance
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,56937.0,101.0,1.0,1.0,3900000.0,19940274.0,54818.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,45971.0,78.0,0.0,0.0,746000.0,2859024.0,48271.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40615.0,83.0,0.0,0.0,664000.0,9408576.0,38141.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,41083.0,72.0,0.0,0.0,1000000.0,4400587.0,44105.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,47943.0,63.0,0.0,0.0,927000.0,4145494.0,46249.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,50144.0,68.0,0.0,0.0,552000.0,3052498.0,27762.0
29031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,43651.0,63.0,0.0,0.0,1300000.0,6330422.0,36935.0
29032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,41915.0,84.0,0.0,0.0,1600000.0,4648486.0,41445.0
29033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,45494.0,100.0,1.0,1.0,1100000.0,2811927.0,44615.0


In [44]:
# Build random forest model - 9673 cases from 2005-2007
modeldata = df_encoded[:9673]

# Define features (X) and target variable (y)
X = modeldata.drop(columns=['attendance'])
y = modeldata['attendance']

# Split data into training / validation / test (80% train, 10% val, 10% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


# Initialize the Random Forest regressor
model = RandomForestRegressor(n_estimators=300, random_state=42, max_depth=None, min_samples_leaf=2, min_samples_split=5)
# Train the model
model.fit(X_train, y_train)

# Predict on the val set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE) and R-squared score (R²)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(1/2)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'RMSE: {rmse:.2f}')
print(f'R-squared Score (R²): {r2:.2f}')

Mean Squared Error (MSE): 23970687.59
RMSE: 4895.99
R-squared Score (R²): 0.81


In [ ]:
# Random Forest hyperparameter tuning
# Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
# Best Parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}


In [60]:
# Build Lasso regression model

X = modeldata.drop(columns=["attendance"])
y = modeldata["attendance"]

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit Lasso Regression
lasso = Lasso(alpha=0.1)  # Adjust alpha to control regularization strength
lasso.fit(X_scaled, y)

# Extract feature importance
feature_significance = pd.DataFrame({
    "Variable": X.columns,
    "Lasso_coef": lasso.coef_
})

# Rank features by absolute coefficient value
feature_significance["Abs_Coefficient"] = np.abs(feature_significance["Lasso_coef"])
feature_significance = feature_significance.sort_values(by="Abs_Coefficient", ascending=False)

# Display ranked features
feature_significance[["Variable", "Lasso_coef"]].head(10)

C:\Users\travi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.677e+10, tolerance: 1.171e+08
  model = cd_fast.enet_coordinate_descent(


,Variable,Lasso_coef
72,cat__h_name_NYY,3423.185821
67,cat__h_name_LAD,2752.576607
120,day_of_week,2609.080110
68,cat__h_name_MIA,-2472.664145
80,cat__h_name_TB,-2288.060932
71,cat__h_name_NYM,2286.315448
103,cat__park_id_Minute Maid Park,2032.529458
65,cat__h_name_KC,-1924.646072
79,cat__h_name_STL,1821.750302
73,cat__h_name_OAK,-1653.856715


In [61]:
# Cross validation for Lasso model
lasso_cv = LassoCV(cv=5).fit(X_scaled, y)
print(f"Optimal alpha: {lasso_cv.alpha_}")

Optimal alpha: 31.640845472570195


In [62]:
# Build Lasso for making predictions
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train Lasso Regression model
lasso = Lasso(alpha=31.640845472570195)
lasso.fit(X_train, y_train)

# Predict on test data
y_pred = lasso.predict(X_test)



lasso_predictions = pd.DataFrame({'Actual':y_test, 'Predictions':y_pred})
lasso_predictions.head()

,Actual,Predictions
5072,13391.0,15266.901196
487,38984.0,38818.654057
1155,23147.0,17374.260037
6422,11556.0,15440.916707
5337,37359.0,30396.653839


In [63]:
# Evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mse**(1/2)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f'RMSE: {rmse:.2f}')
print(f"R² Score: {r2:.4f}")

Mean Squared Error (MSE): 36573746.0116
RMSE: 6047.62
R² Score: 0.7044
